In [1]:
import numpy as np
np.random.seed(13)
from sklearn import svm
import pandas as pd
import matplotlib.pyplot as plt
import math
from __future__ import division
from sklearn.model_selection import KFold
from sklearn.preprocessing import normalize
from stockstats import StockDataFrame 
from googlefinance.client import get_price_data, get_prices_data, get_prices_time_data
import dataPreparation
param = {
    'q': 'ADBE', # Stock symbol (ex: "AAPL")
    'i': "3600", # Interval size in seconds ("86400" = 1 day intervals)
    'x': "NASD", # INDEXNASDAQStock exchange symbol on which stock is traded (ex: "NASD")
    'p': "1Y" # Period (Ex: "1Y" = 1 year)
    }
   
# link to doc http://www.networkerror.org/component/content/article/1-technical-wootness/44-googles-undocumented-finance-api.html
# get price data (return pandas dataframe) 1465948800
priceData = get_price_data(param)
#print(priceData)
stock = StockDataFrame.retype(priceData)
#print(stock)
string=['open_-3_d','open_-6_d','open_-12_d','rsi_3','rsi_6','rsi_12']

#['macd','macds', 'boll_ub', 'boll_lb','rsi_6','rsi_12','vr_6_sma','wr_10','wr_6''close_10_sma''tr''middle_{}_sma'
#df['dma'] = df['close_10_sma'] - df['close_50_sma'] pdi'] = cls._get_pdi(df, 14)df['mdi'] = cls._get_mdi(df, 14)
#df['dx'] = cls._get_dx(df, 14) df['adx'] = df['dx_6_ema'] df['adxr'] = df['adx_6_ema']]
#



stock.get(string)
price=stock
print(price)

/home/andrea/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/andrea/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


                         open      high       low     close  volume  \
2017-05-02 22:00:00  135.5400  135.7000  134.7700  134.7700  227115   
2017-05-02 23:00:00  134.8100  135.2700  134.7500  135.0981  224256   
2017-05-03 00:00:00  135.0900  135.5300  135.0500  135.3750  255306   
2017-05-03 01:00:00  135.3850  135.6300  135.3850  135.4350  137523   
2017-05-03 02:00:00  135.4361  135.4500  135.1400  135.2100  168183   
2017-05-03 03:00:00  135.2000  135.2500  134.8500  134.9350  240585   
2017-05-03 04:00:00  134.9400  135.1100  134.8300  135.0000  743791   
2017-05-03 22:00:00  134.9900  135.2100  134.5300  134.9100  132154   
2017-05-03 23:00:00  134.9500  135.0000  134.3900  134.4500  327036   
2017-05-04 00:00:00  134.4600  134.9500  134.3800  134.9398  191997   
2017-05-04 01:00:00  134.9200  134.9500  134.6900  134.9000   88101   
2017-05-04 02:00:00  134.9000  134.9300  134.6199  134.8200  101402   
2017-05-04 03:00:00  134.7900  134.9300  134.5700  134.7400  167891   
2017-0

In [2]:

xtemp=price[string]
xtemp=np.nan_to_num(np.asarray(xtemp, dtype=float))
xtemp=normalize(xtemp,axis=0,norm='max')
print(xtemp)
accintime=[]
trendwindowtime=[2,6,12,30,60,100]
for t in trendwindowtime:
    #1
#label because of the maket and append values without data
#simo theroy past trend
    x=[]
    y=[]
    print('============================================================')
    print('============================================================')
    print('Working on window:',t)
    print(len(xtemp))
    ##QUI C E L'UNICO APPUNTO GUARDA SE CON +1 CAMBIA
    for i in range(0,len(price)-t-1):
        y.append(np.sign(price.iloc[i+t+1]['close']-price.iloc[i+1]['open']))
        x.append(xtemp[i])
    y=np.array(y)
    x=np.array(x)
    #Split betwwen train-validation and test
    #Faccio random permuatation cosi e come se lo facessi su 5 dataset completamentediversi
    
    
    permindex=range(0,len(x))
    #permindex=np.random.permutation(permindex)
    train=0.8
    nt=math.ceil(len(x)*train)
    trainvalindex=permindex[0:nt]
    testindex=permindex[nt:]
    x_tv=[]
    y_tv=[]
    x_test=[]
    y_test=[]
    x_tv=x[trainvalindex]
    y_tv=y[trainvalindex]
    x_test=x[testindex]
    y_test=y[testindex]

    #Fairly sampling the tv
    posindex=np.where( y_tv > 0 )
    negindex=np.where( y_tv < 0 )

    yindex=[]
    nindex=min(len(posindex[0]),len(negindex[0]))

    #for i in range(1,nindex):
    y_tvnew=np.concatenate((y_tv[posindex[0][0:nindex]],y_tv[negindex[0][0:nindex]]))
    x_tvnew=np.concatenate((x_tv[posindex[0][0:nindex]],x_tv[negindex[0][0:nindex]]))

    #Fairly sampling the test 50% 50%
    posindex=np.where( y_test > 0 )
    negindex=np.where( y_test < 0 )

    yindex=[]
    nindex=min(len(posindex[0]),len(negindex[0]))

    #for i in range(1,nindex):
    y_testnew=np.concatenate((y_test[posindex[0][0:nindex]],y_test[negindex[0][0:nindex]]))
    x_testnew=np.concatenate((x_test[posindex[0][0:nindex]],x_test[negindex[0][0:nindex]]))

    
    nfold=10
    kf = KFold(n_splits=nfold, random_state=13, shuffle=True)


    cspace=np.logspace(-4,4,5)
    gspace=np.logspace(-4,4,5)
    bestsvm=None
    maxacc=0
    cvacc=0
    maxg=0
    maxc=0
    print('Model Selection...')
    #model selection
    for c in cspace:
        #print()
        for g in gspace:
            cvacc=0
            #crossvalidation
            for train_index, test_index in kf.split(x_tvnew):
                x_train, x_val = x_tvnew[train_index], x_tvnew[test_index]
                y_train, y_val = y_tvnew[train_index], y_tvnew[test_index]
                rbf_svm=svm.SVC(kernel='rbf',C=c,gamma=g)
                rbf_svm.fit(x_train,y_train)
                accuracy=rbf_svm.score(x_val,y_val)
                cvacc=cvacc+accuracy/nfold
            #print('ecco c and gamma',c,g)
            #print(cvacc)
            if(cvacc>maxacc):
                maxacc=cvacc
                maxg=g
                maxc=c
                bestsvm=rbf_svm
    
    print('ACCURACY on val:',maxacc)
    print('Gamma:',maxg)
    print('C:',maxc)
    #print(bestsvm)
    rbf_svm=svm.SVC(kernel='rbf',C=maxc,gamma=maxg)
    rbf_svm.fit(x_tvnew,y_tvnew)
    testacc=rbf_svm.score(x_testnew,y_testnew)
    print('Accuracy on TEST set:',testacc)
    accintime.append(testacc)
    
print(accintime)

[[ 0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          1.          1.          1.        ]
 [ 0.          0.          0.          1.          1.          1.        ]
 ...
 [ 0.06303725 -0.04338536  0.01072386  0.73809441  0.60113681  0.54605761]
 [ 0.04918179  0.05233584 -0.04457551  0.86342941  0.7021246   0.60236465]
 [ 0.14963387  0.15016146  0.02815013  0.89510243  0.73587928  0.62377872]]
Working on window: 2
1755
Model Selection...
ACCURACY on val: 0.5500270965993768
Gamma: 100.0
C: 100.0
Accuracy on TEST set: 0.5304878048780488
Working on window: 6
1755
Model Selection...
ACCURACY on val: 0.6109563807839669
Gamma: 100.0
C: 1.0
Accuracy on TEST set: 0.5128205128205128
Working on window: 12
1755
Model Selection...
ACCURACY on val: 0.592150866462793
Gamma: 100.0
C: 1.0
Accuracy on TEST set: 0.49645390070921985
Working on window: 30
1755
Model Selection...
ACCURACY on val: 0.62
Gamma: 100.0
C: 1.0
Accuracy on TEST set: 0.520

In [3]:
AAPL
============================================================
============================================================
Working on window: 2
3517
Model Selection...
ACCURACY on val: 0.5296694046962529
Gamma: 1.0
C: 10000.0
Accuracy on TEST set: 0.5058997050147492
============================================================
============================================================
Working on window: 6
3517
Model Selection...
ACCURACY on val: 0.5478036132485781
Gamma: 100.0
C: 100.0
Accuracy on TEST set: 0.4807692307692308
============================================================
============================================================
Working on window: 12
3517
Model Selection...
ACCURACY on val: 0.5854233712365016
Gamma: 100.0
C: 1.0
Accuracy on TEST set: 0.5243161094224924
============================================================
============================================================
Working on window: 30
3517
Model Selection...
ACCURACY on val: 0.5951320243737305
Gamma: 100.0
C: 1.0
Accuracy on TEST set: 0.4936708860759494
============================================================
============================================================
Working on window: 60
3517
Model Selection...
ACCURACY on val: 0.5873015873015873
Gamma: 1.0
C: 10000.0
Accuracy on TEST set: 0.45871559633027525
============================================================
============================================================
Working on window: 100
3517
Model Selection...
ACCURACY on val: 0.6013422818791946
Gamma: 100.0
C: 1.0
Accuracy on TEST set: 0.5
[0.5058997050147492, 0.4807692307692308, 0.5243161094224924, 0.4936708860759494, 0.45871559633027525, 0.5]
ADBE
============================================================
============================================================
Working on window: 2
3517
Model Selection...
ACCURACY on val: 0.5547935243154367
Gamma: 100.0
C: 100.0
Accuracy on TEST set: 0.5015923566878981
============================================================
============================================================
Working on window: 6
3517
Model Selection...
ACCURACY on val: 0.5752002431523994
Gamma: 100.0
C: 1.0
Accuracy on TEST set: 0.5267558528428093
============================================================
============================================================
Working on window: 12
3517
Model Selection...
ACCURACY on val: 0.5915799167200513
Gamma: 100.0
C: 1.0
Accuracy on TEST set: 0.5641509433962264
============================================================
============================================================
Working on window: 30
3517
Model Selection...
ACCURACY on val: 0.5708691598466797
Gamma: 100.0
C: 1.0
Accuracy on TEST set: 0.5316455696202531
============================================================
============================================================
Working on window: 60
3517
Model Selection...
ACCURACY on val: 0.5763579638752053
Gamma: 100.0
C: 100.0
Accuracy on TEST set: 0.49122807017543857
Working on window: 100
3517
Model Selection...
ACCURACY on val: 0.6112328767123288
Gamma: 100.0
C: 1.0
Accuracy on TEST set: 0.4061032863849765
[0.5015923566878981, 0.5267558528428093, 0.5641509433962264, 0.5316455696202531, 0.49122807017543857, 0.4061032863849765]
​

SyntaxError: invalid syntax (<ipython-input-3-5db4e4fb296c>, line 2)